In [1]:
using Distributed
@everywhere using StaticArrays,BenchmarkTools,Random,Flux,DelimitedFiles,LinearAlgebra
using InvertedIndices
@everywhere using MachineLearningPotential

In [2]:
Threads.nthreads()

8

# Symmetry Functions

Time to write and test the basics of SymmFunctions for ML potentials

In [3]:
atoms = [[0.0000006584,       -0.0000019175,        0.0000000505],
[-0.0000005810,       -0.0000004871,        0.6678432175],
[0.1845874248,       -0.5681026047,        0.2986701538],
[-0.4832557457,       -0.3511072166,        0.2986684497],
[-0.4832557570,        0.3511046452,        0.2986669456],
[0.1845874064,        0.5681000550,        0.2986677202],
[0.5973371920,       -0.0000012681,        0.2986697030],
[-0.1845860897,       -0.5681038901,       -0.2986676192],
[-0.5973358752,       -0.0000025669,       -0.2986696020],
[-0.1845861081,        0.5680987696,       -0.2986700528],
[0.4832570624,        0.3511033815,       -0.2986683486],
[0.4832570738,       -0.3511084803,       -0.2986668445],
[0.0000018978,       -0.0000033480,       -0.6678431165],
[-0.0000017969,        0.0000009162,        1.3230014650],
[0.1871182835,       -0.5758942175,        0.9797717078],
[-0.4898861924,       -0.3559221410,       0.9797699802],
[-0.4898862039,        0.3559224872,        0.9797684555],
[0.1871182648,        0.5758945856,        0.9797692407],
[0.6055300485,        0.0000001908,        0.9797712507],
[0.7926501864,       -0.5758950093,        0.6055339635],
[0.3656681761,       -1.1254128670,        0.5916673591],
[-0.3027660545,       -0.9318173412,        0.6055326929],
[-0.9573332453,       -0.6955436707,        0.5916639831],
[-0.9797705418,       -0.0000006364,        0.6055294407],
[-0.9573332679,        0.6955423392,        0.5916610035],
[-0.3027660847,        0.9318160902,        0.6055287012],
[0.3656681396,        1.1254115783,        0.5916625380],
[0.7926501677,        0.5758937939,        0.6055314964],
[1.1833279992,       -0.0000006311,        0.5916664660],
[0.6770051458,       -0.9318186223,        0.0000033028],
[0.0000006771,       -1.1517907207,        0.0000025175],
[-0.6770037988,       -0.9318186442,        0.0000007900],
[-1.0954155825,       -0.3559242494,       -0.0000012200],
[-1.0954155940,        0.3559203788,       -0.0000027447],
[-0.6770038290,        0.9318147872,       -0.0000032017],
[0.0000006397,        1.1517868856,       -0.0000024165],
[0.6770051155,        0.9318148091,       -0.0000006889],
[1.0954168993,        0.3559204143,        0.0000013211],
[1.0954169108,       -0.3559242139,        0.0000028458],
[0.3027674014,       -0.9318199253,       -0.6055286002],
[-0.3656668229,       -1.1254154134,       -0.5916624370],
[-0.7926488510,       -0.5758976290,       -0.6055313954],
[-1.1833266824,       -0.0000032040,       -0.5916663649],
[-0.7926488697,        0.5758911742,       -0.6055338624],
[-0.3656668594,        1.1254090319,       -0.5916672580],
[0.3027673712,        0.9318135061,       -0.6055325919],
[0.9573345621,        0.6955398357,       -0.5916638820],
[0.9797718586,       -0.0000031986,       -0.6055293396],
[0.9573345846,       -0.6955461743,       -0.5916609025],
[-0.1871169480,       -0.5758984207,       -0.9797691397],
[-0.6055287318,       -0.0000040259,       -0.9797711497],
[-0.1871169667,        0.5758903824,       -0.9797716067],
[0.4898875091,        0.3559183059,       -0.9797698792],
[0.4898875207,       -0.3559263223,       -0.9797683545],
[0.0000031136,       -0.0000047513,       -1.3230013639]]*18.8973*0.36258

positions = [SVector{3}(p[i] for i in 1:3) for p in atoms]
dis2mat = get_distance2_mat(positions)
rad_function = RadialType2{Float64}(0.001,11.338,[1.,1.])
ang_func = AngularType3{Float64}(0.0001,1.0,1.0,11.338,[1.,1.,1.])
ang_func2 = AngularType3{Float64}(0.0001,-1.0,2.0,11.338,[1.,1.,1.])

AngularType3{Float64}(0.0001, -1.0, 2.0, 11.338, [1.0, 1.0, 1.0])

Demonstrates the difference between [:,index] and [index,:]

# Angular Symmetry Function

Testing prior to type system

In [4]:
@benchmark calc_symmetry_function($positions,$dis2mat,rand(1:55),rad_function)

BenchmarkTools.Trial: 10000 samples with 200 evaluations.
 Range (min … max):  389.030 ns …  18.463 μs  ┊ GC (min … max): 0.00% … 95.90%
 Time  (median):     451.028 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   552.057 ns ± 763.487 ns  ┊ GC (mean ± σ):  6.64% ±  4.68%

   ▁▄▆███▇▅▄▃    ▁▃▂▂▁▁      ▁   ▂▄▅▆▆▆▆▆▅▄▄▃▂▁▁▂▁▁▁▁▁▁         ▃
  ▃███████████▇▆████████▇▆▆███████████████████████████████▇███▇ █
  389 ns        Histogram: log(frequency) by time        754 ns <

 Memory estimate: 1.06 KiB, allocs estimate: 4.

In [5]:
@benchmark calc_symmetry_function($positions,$dis2mat,rand(1:55),ang_func)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  24.053 μs …  2.459 ms  ┊ GC (min … max): 0.00% … 97.97%
 Time  (median):     32.069 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   32.907 μs ± 29.614 μs  ┊ GC (mean ± σ):  1.21% ±  1.37%

              ▃▇█▇▃   ▁▃▄▄▁                                    
  ▁▁▄▇█▅▄▂▃▄▆▆██████▅▇████████▇▆▅▃▄▄▄▅▅▄▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁ ▃
  24.1 μs         Histogram: frequency by time        47.2 μs <

 Memory estimate: 22.02 KiB, allocs estimate: 9.

In [6]:
calc_symmetry_function(positions,dis2mat,1,ang_func)

35.92620108172107

In [7]:
using Distributed
Base.Threads.nthreads()

8

In [8]:
@everywhere import MachineLearningPotential.SymmetryFunctions.calc_one_symm_func


In [9]:
@everywhere function calc_thr_symm_func(positions,dis2_mat,index,symmfunc::AngularType3)
    g_vec = []
    #g_vec = [calc_one_symm_func(positions[index],positions[j],positions[k],dis2_mat[index,j],dis2_mat[index,k],dis2_mat[j,k],symmfunc) for j=(1:55) if j !=index for k= 1:j-1 if k != index]
    for j=(1:55)
        if j != index
            for k  = (1:j-1)# if k!=index)
                if k!= index
                    push!(g_vec, calc_one_symm_func(positions[index],positions[j],positions[k],dis2_mat[index,j],dis2_mat[index,k],dis2_mat[j,k],symmfunc) )
                end            
            end
        end
    end

    
    return sum(g_vec)*2^(1-symmfunc.zeta)
end
@benchmark calc_thr_symm_func($positions,$dis2mat,rand(1:55),ang_func)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  38.851 μs …  2.166 ms  ┊ GC (min … max): 0.00% … 95.91%
 Time  (median):     50.450 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   53.777 μs ± 53.715 μs  ┊ GC (mean ± σ):  2.60% ±  2.56%

               ▃▆█▆▄       ▁▂▂▁                                
  ▁▁▂▃▆▇▇▅▄▃▂▃▇██████▆▅▄▃▄▆████▇▆▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁ ▃
  38.9 μs         Histogram: frequency by time        75.5 μs <

 Memory estimate: 66.58 KiB, allocs estimate: 2869.

In [11]:
calc_thr_symm_func(positions,dis2mat,1,ang_func)
@everywhere function calc_thr_symm_func2(positions,dis2_mat,index,symmfunc::AngularType3)
    g_vec = zeros(1485)
    for j=(1:55)
        if j != index
            for k  = (1:j-1)# if k!=index)
                if k!= index
                    ind = Int((j^2 - 3j)/2 + 1 + k)
                    g_vec[ind] = calc_one_symm_func(positions[index],positions[j],positions[k],dis2_mat[index,j],dis2_mat[index,k],dis2_mat[j,k],symmfunc) 
                end            
            end
        end
    end

    
    return sum(g_vec)*2^(1-symmfunc.zeta)
end
calc_thr_symm_func2(positions,dis2mat,1,ang_func)
@benchmark calc_thr_symm_func2($positions,$dis2mat,rand(1:55),ang_func)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  21.264 μs …  2.436 ms  ┊ GC (min … max): 0.00% … 97.98%
 Time  (median):     26.913 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   27.943 μs ± 24.376 μs  ┊ GC (mean ± σ):  0.85% ±  0.98%

     ▁              ▄█▆                                        
  ▁▃▅█▇▄▂▂▂▃▄▃▂▁▂▂▄▇████▆▅▅▆▆▅▄▂▂▂▂▂▂▃▃▄▆▆▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁ ▃
  21.3 μs         Histogram: frequency by time        37.3 μs <

 Memory estimate: 11.77 KiB, allocs estimate: 2.

In [12]:
@everywhere function calc_thr_symm_func3(positions,dis2_mat,index,symmfunc::AngularType3)
    g_vec = zeros(1485)
    Threads.@threads for j=(1:55)
        if j != index
            for k  = (1:j-1)# if k!=index)
                if k!= index
                    ind = Int((j^2 - 3j)/2 + 1 + k)
                    g_vec[ind] = calc_one_symm_func(positions[index],positions[j],positions[k],dis2_mat[index,j],dis2_mat[index,k],dis2_mat[j,k],symmfunc) 
                end            
            end
        end
    end

    
    return sum(g_vec)*2^(1-symmfunc.zeta)
end

In [13]:
calc_thr_symm_func3(positions,dis2mat,1,ang_func)

35.926201081721075

In [14]:
@benchmark calc_thr_symm_func3($positions,$dis2mat,rand(1:55),ang_func)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   8.295 μs …  2.792 ms  ┊ GC (min … max): 0.00% … 98.39%
 Time  (median):     11.644 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   12.249 μs ± 27.977 μs  ┊ GC (mean ± σ):  2.24% ±  0.98%

            ▂▄▆▇▆█▆▇▇▇▆▅▄▅▅▅▆▄▄▄▃▂                             
  ▁▁▂▂▃▄▅▇▇████████████████████████▇▆▆▅▅▄▄▃▄▄▃▂▃▂▂▂▂▂▁▁▁▁▁▁▁▁ ▅
  8.3 μs          Histogram: frequency by time        17.5 μs <

 Memory estimate: 16.50 KiB, allocs estimate: 51.